## Constrained docking protocol

In this tutorial, we will demonstrate how you can use `rush-py` to conduct a large-scale virtual screen on a target using a constrained docking protocol.

We will use the Zinc20 database of FDA approved drugs as our sample ligand database, but Rush's capability means that this protocol could scale to screen tens of millions of ligands.

## 0.0) Imports

In [20]:
import rdkit
import os 
import os
import math
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem import MolFromSmiles, MolFromSmarts, SDMolSupplier
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS, rdRascalMCES
from rdkit.Chem import rdDistGeom
from rdkit import Chem
from rdkit.Chem.rdMolAlign import AlignMol
from rdkit.Chem import rdForceFieldHelpers

In [3]:
def constrained_docking(): pass

In [21]:
def find_mcs(query_ligands, reference, timeout=3600, **kwargs):
    if kwargs.get("ignore_heavy_atom"):
        atom_comparision_method = rdFMCS.AtomCompare.CompareAnyHeavyAtom
    else:
        atom_comparision_method = rdFMCS.AtomCompare.CompareAnyHeavyAtom

    if kwargs.get("ignore_bond_order"):
        bond_comparision_method = rdFMCS.BondCompare.CompareAny
    else:
        bond_comparision_method = rdFMCS.BondCompare.CompareOrder

    mcs_results = []

    for query_ligand in query_ligands:
        if kwargs.get("use_rascal_mces"):
            mcs_results.append(
                rdRascalMCES.FindMCES(
                    [reference, query_ligand],
                    atomCompare=atom_comparision_method,
                    bondCompare=bond_comparision_method
                )
            )
        else:
            mcs_results.append(rdFMCS.FindMCS(
                [reference, query_ligand],
                threshold=0.9,
                completeRingsOnly=kwargs.get("complete_rings_only", False),
                atomCompare=atom_comparision_method,
                bondCompare=bond_comparision_method,
                timeout=timeout
            )
            )

    return mcs_results


def meets_similarity_threshold(mcs, reference, min_threshold=0.2):
    if mcs_result_exists(mcs):
        mcs_mol = Chem.MolFromSmarts(mcsResult.smartsString, mergeHs=True)
        match_ratio = mcs_mol.GetNumAtoms() / reference.GetNumAtoms()

        return match_ratio >= min_threshold
    
    return False


def filter_mcs(mcss, reference):
    return [mcs for mcs in mcss if meets_similarity_threshold(mcs, reference)]


INITIAL_FILTER_DISTANCE = 1000
def get_diverse_substructure_hits(reference, mcs, minimum_difference=5):
    """
    Prunes a list of MCS substructure hits within a molecule and keeps only those which are at least the minimum difference apart
    This is to capture as many unique substructure hits across the query and the ligand, but discard those that are not meaningfully different (e.g. a rotation of the bond) 
    A default of 5 is an effective balance between retaining novel hits   
    """
    mcs_molecule = MolFromSmarts(mcsResult.smartsString, mergeHs=True)
    substructures = reference.GetSubstructMatches(mcs_molcule, uniquify=False)

    output_structures = [
        substructures[0]
    ]
    for substructure in substructures[1:]:
        distance = INITIAL_FILTER_DISTANCE
        j = 0

        while (distance >= minimum_difference) and j < len(output_structures):
            ref = np.array(output_structures[j])
            distance = sum(np.array(substructure) != ref)
            j += 1

        if distance >= mindiff:
            output_structures.append(substructure)

    return output_structures


def get_tethered_atoms(substruct_match) -> str:
    """
    Return a formatted string of atom indexes to pass to "TETHERED ATOMS" configuration option for rxdock

    Example input:
    (0, 1, 2)

    Example output:
    1,2,3
    """
    return ','.join(str(index + 1) for index in substruct_match)

def mcs_result_exists(mcs) -> bool:
    return mcs.smartsString and len(mcs.smartsString) > 0


def get_force_field(geom_calc, query_mol):
    ff = rdForceFieldHelpers.UFFGetMoleculeForceField(query_mol, confId=0)

    for i in geom_calc.coordMap:
        point = geom_calc.coordMap[i]
        point_idx = ff.AddExtraPoint(point.x, point.y, point.z, fixed=True) - 1
        ff.AddDistanceConstraint(point_idx, i, 0, 0, 100.0)
    ff.Initialize()

    return query_mol


def constrained_embed(query, reference, geom_calc, n_tries=5) -> Chem.Mol:
    """
    Align query molecule to "template" (reference molecule)
    Distance constrints are enforced on the input ligand to mould the geometry of its matching substructure to that in the template
    """
    temp_query_mol = Chem.Mol(query) 

    min_energy = np.inf
    best_mol = None

    ff = get_forcefield(geom_calc, temp_query_mol)

    AlignMol(temp_query_mol, reference, atomMap=algMap)


   
def get_substructure_matches(molecule, mcs) -> Chem.Mol:
    mcs_mol = Chem.MolFromSmarts(mcsResult.smartsString, mergeHs=True)
    return molecule.GetSubstructMatches(mcs_mol)


def get_dist_geom_calculator(reference, query_match, reference_match, constrained_atoms):
    geom_calc = rdDistGeom.ETKDGv3()
    geom_calc.trackFailures = True

    geom_calc.coordMap = {
        query_match[atom_idx]: reference.GetConformer().GetAtomPosition(reference_match[atom_idx])for atom_idx in range(len(constrained_atoms))
    }

    return geom_calc




In [7]:
# test find_mcs
reference = MolFromSmiles("CCO")
query = MolFromSmiles("CC(C)O")
mcs = find_mcs([query], reference)

mcs[0].smartsString

'[#6]-[#6]-[#8,#6]'

In [9]:
# test filter_varied_mcs
patt = MolFromSmarts(mcs[0].smartsString, mergeHs=True)
substructures = reference.GetSubstructMatches(patt, uniquify=False)

In [17]:
ligfilename = 'LIA_ideal.sdf' # This is the corresponding sdf file of the template ligand. Missing hydrogens are ok here
reference = SDMolSupplier(ligfilename, sanitize=False)[0] # The hydrogens are dropped in this step, which is convenient, since we do not want H-alignment in the substructure matching step
